In [1]:
%load_ext autoreload
%autoreload 2
from llm_utils import limp_llm_plan, build_intent_dict, generate_json_with_gpt

## Example From David

In [ ]:
## Test instructions:
"Pick up the book on the table with the red cover."
"Place the vase near the flowers on the table."
"Place the coke can beside the pringles on the counter."
"Bring the book and the magazine on the nightstand."
"Bring the mug from the table near the sink."

In [3]:
## Test example
# Transcript: place the coke can beside the pringles on the counter

# interpretation_1
# "Place the coke can (which is currently beside the pringles) on some counter."
intent_labels_1 = {"place": 0, "the": 0, "coke": 1, "can": 1, "beside": 0, "the": 0, "pringles": 2, "on": 0, "the": 0, "counter": 1}


# interpretation_2
# "Take the coke can (which is somewhere else) and place it beside the pringles (that are already on the counter).",
intent_labels_2 = {"place": 0,"the": 0,"coke": 1,"can": 1,"beside": 0,"the": 0,"pringles": 1,"on": 0,"the": 0,"counter": 2}


In [7]:
instruction = input("Enter instruction: ")

In [5]:
passedin_intent_dict = {'Goal_intent': ['counter'],
 'Avoidance_intent': [],
 'Detail_intent': ['coke', 'can']}

In [ ]:
baseline_choice = "intent_label"
passedin_intent_dict = build_intent_dict(intent_labels_1)
# baseline_choice = "asr"

baseline_prompt_path = f"prompts/{baseline_choice}_prompt.txt"


limp_llmplaner_incontext_prompt = open(baseline_prompt_path, "r").read() 
limp_planner_output = limp_llm_plan(limp_llmplaner_incontext_prompt,instruction, passedin_intent_dict, baseline=baseline_choice, verbose=False)

print("**************************************************\nInstruction:",instruction)
print(f"--------------------------------------------------\nLLM Planner Output\n--------------------------------------------------\n{limp_planner_output}\n**************************************************")


In [ ]:
json_output = generate_json_with_gpt(limp_planner_output)

print("**************************************************\nJson Output:")

# print json in tree format
import json
print(json.dumps(json_output, indent=4, sort_keys=True))

## Util Functions

In [4]:
import json

def build_task_plans(model_output_file, task_plan_file):
    """
    Reads model outputs, generates structured task plans, and writes them to a JSON file.

    Parameters:
    - model_output_file (str): Path to the input JSON file containing model outputs.
    - task_plan_file (str): Path to the output JSON file where task plans will be saved.
    """

    # Indicate the start of the function
    print("Building task plans...")

    # Initialize a dictionary to store task plans keyed by audio file names
    task_plans = {}

    # Define experiment baseline
    baseline_choice = "intent_label"
    baseline_prompt_path = f"prompts/{baseline_choice}_prompt.txt"

    # Read the prompt from the specified file
    try:
        with open(baseline_prompt_path, "r", encoding='utf-8') as prompt_file:
            limp_llmplaner_incontext_prompt = prompt_file.read()
    except FileNotFoundError:
        print(f"Prompt file not found at {baseline_prompt_path}. Please check the path.")
        return
    except Exception as e:
        print(f"An error occurred while reading the prompt file: {e}")
        return

    # Read the model output JSON file
    try:
        with open(model_output_file, encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Model output file not found at {model_output_file}. Please check the path.")
        return
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {model_output_file}: {e}")
        return
    except Exception as e:
        print(f"An error occurred while reading the model output file: {e}")
        return

    # Iterate over each entry in the JSON data
    for entry in data:
        try:
            # Extract required fields
            audio_file = entry['audio_file']
            instruction = entry['sentence']
            intent_dict = entry['intent_dict']
        except KeyError as e:
            print(f"Missing expected key {e} in entry: {entry}")
            continue

        # Generate the task plan using the limp_llm_plan function
        try:
            limp_planner_output = limp_llm_plan(
                limp_llmplaner_incontext_prompt,
                instruction,
                intent_dict,
                baseline=baseline_choice,
                verbose=False
            )
        except Exception as e:
            print(f"Error generating task plan for {audio_file}: {e}")
            continue

        # Generate JSON version of the task plan using the generate_json_with_gpt function
    
        try:
            task_plan_json_str = generate_json_with_gpt(limp_planner_output)
        except Exception as e:
            print(f"Error generating JSON for task plan of {audio_file}: {e}")
            continue

        # Parse the JSON string into a Python dictionary to avoid storing it as a string
        try:
            task_plan_dict = json.loads(task_plan_json_str)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON task plan for {audio_file}: {e}")
            print(f"Task Plan String: {task_plan_json_str}")
            continue

        # Store the task plan in the task_plans dictionary with audio_file as the key
        task_plans[audio_file] = {
            'instruction': instruction,
            'task_plan': task_plan_dict  # Store as a nested dictionary
        }

    # Write the task_plans dictionary to the output JSON file
    try:
        with open(task_plan_file, 'w', encoding='utf-8') as f:
            json.dump(task_plans, f, ensure_ascii=False, indent=4, sort_keys=True)
    except Exception as e:
        print(f"Error writing to {task_plan_file}: {e}")
        return

    # Indicate the completion of the function
    print("Task plans built and saved to file.")


In [6]:
# running for Bilstm model
model_output_file = "../ltl/data/filtered_prosody_transformer_multiclass_results.json"
# model_output_file = "../ltl/data/test_sample.json"
task_plan_file = "../ltl/data/transformer_generated_task_plans.json"

build_task_plans(model_output_file, task_plan_file)

Building task plans...
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: gpt-4o || Deterministic?: True
Executing Baseline: intent_label
		Model: gpt-4 || Deterministic?: True
		Model: 